# 电力变压器油温预测 - RNN模型# Power Transformer Oil Temperature Prediction - RNN本notebook使用循环神经网络（RNN）模型预测变压器油温。**主要特性 / Key Features:**- 使用共享 `utils.py` 模块，代码简洁- RNN能够捕获时间序列的顺序依赖关系- 支持单向/双向RNN、多层堆叠- 一次训练一个模型，参数可配置- 支持三种数据分割方式和特征选择

In [ ]:
# 导入所需库 / Import Required Librariesimport numpy as npimport torchimport torch.nn as nnfrom sklearn.preprocessing import StandardScalerfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_scoreimport warningswarnings.filterwarnings('ignore')# 导入共享工具模块 / Import shared utilitiesfrom utils import *# 设置随机种子 / Set random seedsnp.random.seed(42)torch.manual_seed(42)print(f"默认设备 / Default device: {DEFAULT_CONFIG['device']}")

In [ ]:
# RNN特定配置 / RNN-specific Configuration# 扩展默认配置，添加RNN特定参数RNN_CONFIG = DEFAULT_CONFIG.copy()RNN_CONFIG.update({    # RNN特定参数 / RNN-specific parameters    'hidden_size': 64,      # RNN隐藏层大小    'num_layers': 2,        # RNN层数    'bidirectional': False, # 是否使用双向RNN    'dropout': 0.2,         # Dropout率})print("RNN配置已加载 / RNN configuration loaded")print(f"Hidden size: {RNN_CONFIG['hidden_size']}")print(f"Num layers: {RNN_CONFIG['num_layers']}")print(f"Bidirectional: {RNN_CONFIG['bidirectional']}")

In [ ]:
# RNN模型定义 / RNN Model Definitionclass RNNModel(nn.Module):    """    循环神经网络模型 / Recurrent Neural Network Model    Architecture:    - RNN层（可选多层、双向）    - Dropout    - 全连接输出层    """    def __init__(self, input_size, hidden_size=64, num_layers=2,                 dropout=0.2, bidirectional=False):        super(RNNModel, self).__init__()        self.input_size = input_size        self.hidden_size = hidden_size        self.num_layers = num_layers        self.bidirectional = bidirectional        # RNN层        self.rnn = nn.RNN(            input_size=input_size,            hidden_size=hidden_size,            num_layers=num_layers,            batch_first=True,            dropout=dropout if num_layers > 1 else 0,            bidirectional=bidirectional        )        # Dropout层        self.dropout = nn.Dropout(dropout)        # 全连接输出层        # 如果是双向RNN，输出维度需要乘以2        fc_input_size = hidden_size * 2 if bidirectional else hidden_size        self.fc = nn.Linear(fc_input_size, 1)    def forward(self, x):        """        x shape: (batch, seq_length, input_size)        """        # RNN前向传播        # rnn_out shape: (batch, seq_length, hidden_size * num_directions)        # hidden shape: (num_layers * num_directions, batch, hidden_size)        rnn_out, hidden = self.rnn(x)        # 取最后一个时间步的输出        # shape: (batch, hidden_size * num_directions)        last_output = rnn_out[:, -1, :]        # Dropout        last_output = self.dropout(last_output)        # 全连接层        # shape: (batch, 1)        output = self.fc(last_output)        return outputprint("RNNModel 已定义 / RNNModel defined")

In [ ]:
# RNN训练函数 / RNN Training Functiondef train_rnn_model(train_loader, test_loader, input_size,                    hidden_size=64, num_layers=2, dropout=0.2,                    bidirectional=False, num_epochs=100,                    lr=0.001, patience=10, device='cpu'):    """    训练RNN模型 / Train RNN Model    注意：每个epoch都会打印训练进度 / Note: Prints progress every epoch    """    # 初始化模型    model = RNNModel(        input_size=input_size,        hidden_size=hidden_size,        num_layers=num_layers,        dropout=dropout,        bidirectional=bidirectional    ).to(device)    criterion = nn.MSELoss()    optimizer = torch.optim.Adam(model.parameters(), lr=lr)    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(        optimizer, mode='min', factor=0.5, patience=5, verbose=False    )    # 训练历史    history = {'train_loss': [], 'test_loss': [], 'train_mae': [], 'test_mae': []}    best_test_loss = float('inf')    patience_counter = 0    for epoch in range(num_epochs):        # 训练模式        model.train()        train_losses = []        train_maes = []        for X_batch, y_batch in train_loader:            X_batch, y_batch = X_batch.to(device), y_batch.to(device)            optimizer.zero_grad()            outputs = model(X_batch)            loss = criterion(outputs, y_batch)            loss.backward()            optimizer.step()            train_losses.append(loss.item())            train_maes.append(torch.mean(torch.abs(outputs - y_batch)).item())        # 评估模式        model.eval()        test_losses = []        test_maes = []        with torch.no_grad():            for X_batch, y_batch in test_loader:                X_batch, y_batch = X_batch.to(device), y_batch.to(device)                outputs = model(X_batch)                loss = criterion(outputs, y_batch)                test_losses.append(loss.item())                test_maes.append(torch.mean(torch.abs(outputs - y_batch)).item())        # 计算平均指标        train_loss = np.mean(train_losses)        test_loss = np.mean(test_losses)        train_mae = np.mean(train_maes)        test_mae = np.mean(test_maes)        history['train_loss'].append(train_loss)        history['test_loss'].append(test_loss)        history['train_mae'].append(train_mae)        history['test_mae'].append(test_mae)        # 每个epoch打印 / Print every epoch        print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}')        # 学习率调度        scheduler.step(test_loss)        # Early stopping        if test_loss < best_test_loss:            best_test_loss = test_loss            patience_counter = 0            best_model_state = model.state_dict().copy()        else:            patience_counter += 1            if patience_counter >= patience:                print(f'Early stopping触发，在epoch {epoch+1} / Early stopping at epoch {epoch+1}')                model.load_state_dict(best_model_state)                break    return model, historyprint("RNN训练函数已定义 / RNN training function defined")

In [ ]:
# 主训练函数 / Main Training Functiondef train_single_rnn_model(config=None, **kwargs):    """    训练单个RNN模型 / Train a single RNN model    Parameters:    -----------    config : dict (配置字典，None时使用RNN_CONFIG)    **kwargs : 可覆盖config中的任意参数    Returns:    --------    results : dict 包含模型、历史、评估指标等    """    # 合并配置    if config is None:        config = RNN_CONFIG.copy()    else:        config = config.copy()    config.update(kwargs)    # 获取预测时间范围配置    horizon = config['prediction_horizon']    horizon_cfg = HORIZON_CONFIGS[horizon]    offset = horizon_cfg['offset']    seq_length = config.get('seq_length', horizon_cfg['seq_length'])    print(f"\n{'='*60}")    print(f"RNN训练配置 / RNN Training Configuration:")    print(f"  数据集 / Dataset: {config['dataset_path']}")    print(f"  预测范围 / Horizon: {horizon} ({horizon_cfg['description']})")    print(f"  分割方式 / Split: {config['split_method']}")    print(f"  特征数 / Features: {len(config['load_features']) + len(config['time_features'])}")    print(f"  RNN Hidden: {config['hidden_size']}, Layers: {config['num_layers']}")    print(f"  Bidirectional: {config['bidirectional']}")    print(f"{'='*60}\n")    # 1. 加载数据    df = load_data(config['dataset_path'])    print(f"数据加载完成 / Data loaded: {df.shape}")    # 2. 异常值处理（可选）    if config['remove_outliers']:        df, n_removed = remove_outliers(            df,            method=config['outlier_method'],            threshold=config['outlier_threshold']        )        print(f"移除异常值 / Removed outliers: {n_removed} 行")    # 3. 特征选择    # 3a. 负载特征    load_cols = config['load_features']    X_load = df[load_cols].values    # 3b. 时间特征（可选）    if config['time_features']:        time_features_df = extract_time_features(df, config['time_features'])        X_time = time_features_df.values        X = np.concatenate([X_load, X_time], axis=1)        print(f"特征组合 / Features: {len(load_cols)} 负载 + {len(config['time_features'])} 时间")    else:        X = X_load        print(f"使用负载特征 / Load features: {len(load_cols)}")    # 目标变量    y = df['OT'].values    # 4. 创建序列    X_seq, y_seq = create_sequences_with_offset(X, y, seq_length, offset)    print(f"序列创建完成 / Sequences created: {X_seq.shape}")    # 5. 分割训练/测试集    split_method = config['split_method']    if split_method == 'sequential':        X_train, X_test, y_train, y_test = split_sequential(            X_seq, y_seq, config['train_ratio']        )    elif split_method == 'random':        X_train, X_test, y_train, y_test = split_random_groups(            X_seq, y_seq, config['n_groups'], config['train_ratio']        )    elif split_method == 'label_random':        X_train, X_test, y_train, y_test = split_label_random(            X_seq, y_seq, config['train_ratio']        )    else:        raise ValueError(f"未知的分割方式: {split_method}")    print(f"数据分割完成 / Data split: Train {len(X_train)}, Test {len(X_test)}")    # 6. 归一化（先分割后归一化，避免数据泄露）    scaler_X = StandardScaler()    scaler_y = StandardScaler()    # 展平进行归一化    n_train, seq_len, n_feat = X_train.shape    n_test = X_test.shape[0]    X_train_flat = X_train.reshape(-1, n_feat)    X_test_flat = X_test.reshape(-1, n_feat)    X_train_scaled = scaler_X.fit_transform(X_train_flat).reshape(n_train, seq_len, n_feat)    X_test_scaled = scaler_X.transform(X_test_flat).reshape(n_test, seq_len, n_feat)    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()    print("归一化完成 / Normalization done")    # 7. 创建DataLoader    train_loader, test_loader = create_dataloaders(        X_train_scaled, y_train_scaled,        X_test_scaled, y_test_scaled,        config['batch_size']    )    # 8. 训练RNN模型    print(f"\n开始训练RNN / Start RNN training...\n")    model, history = train_rnn_model(        train_loader, test_loader,        input_size=n_feat,        hidden_size=config['hidden_size'],        num_layers=config['num_layers'],        dropout=config['dropout'],        bidirectional=config['bidirectional'],        num_epochs=config['num_epochs'],        lr=config['learning_rate'],        patience=config['patience'],        device=config['device']    )    # 9. 评估模型    model.eval()    with torch.no_grad():        X_test_t = torch.FloatTensor(X_test_scaled).to(config['device'])        y_pred_scaled = model(X_test_t).cpu().numpy()    # 反归一化    y_pred = scaler_y.inverse_transform(y_pred_scaled).ravel()    # 计算指标    r2 = r2_score(y_test, y_pred)    mse = mean_squared_error(y_test, y_pred)    mae = mean_absolute_error(y_test, y_pred)    print(f"\n{'='*60}")    print(f"最终评估结果 / Final Evaluation:")    print(f"  R² Score: {r2:.6f}")    print(f"  MSE: {mse:.6f}")    print(f"  MAE: {mae:.6f}")    print(f"{'='*60}\n")    # 返回结果    results = {        'model': model,        'history': history,        'scalers': {'X': scaler_X, 'y': scaler_y},        'metrics': {'r2': r2, 'mse': mse, 'mae': mae},        'data': {            'X_train': X_train, 'X_test': X_test,            'y_train': y_train, 'y_test': y_test,            'y_pred': y_pred        },        'config': config    }    return resultsprint("RNN主训练函数已定义 / RNN main training function defined")

## 使用示例 / Usage Examples### 示例1：使用默认RNN配置训练```pythonresults = train_single_rnn_model()```### 示例2：使用双向RNN```pythonresults = train_single_rnn_model(bidirectional=True)```### 示例3：更深的RNN网络```pythonresults = train_single_rnn_model(    num_layers=3,    hidden_size=128)```### 示例4：预测1天的油温```pythonresults = train_single_rnn_model(    prediction_horizon='day')```### 示例5：添加时间特征```pythonresults = train_single_rnn_model(    time_features=['hour', 'dayofweek', 'is_weekend'])```

In [ ]:
# 示例：使用默认RNN配置训练 / Example: Train with default RNN configresults_rnn = train_single_rnn_model()

In [ ]:
# 可视化训练历史 / Visualize Training Historyplot_training_history(results_rnn['history'], 'RNN - Default Config')

In [ ]:
# 可视化预测结果 / Visualize Predictionsplot_predictions(    results_rnn['data']['y_test'],    results_rnn['data']['y_pred'],    f"RNN - {results_rnn['config']['prediction_horizon'].upper()}")

In [ ]:
# 对比实验：不同RNN配置 / Comparison: Different RNN Configurationsprint("\n" + "="*80)print("对比实验：不同RNN配置对模型性能的影响")print("Comparison: Impact of different RNN configurations")print("="*80 + "\n")# 简化配置用于快速对比quick_config = RNN_CONFIG.copy()quick_config['num_epochs'] = 30  # 减少epochs用于快速测试# 测试不同的RNN配置rnn_configs = {    'RNN_1layer': {'num_layers': 1, 'hidden_size': 64, 'bidirectional': False},    'RNN_2layers': {'num_layers': 2, 'hidden_size': 64, 'bidirectional': False},    'BiRNN': {'num_layers': 2, 'hidden_size': 64, 'bidirectional': True},}comparison_results = {}for config_name, config_params in rnn_configs.items():    print(f"\n{'*'*60}")    print(f"测试配置 / Testing config: {config_name}")    print(f"{'*'*60}")    results = train_single_rnn_model(        config=quick_config,        **config_params    )    comparison_results[config_name] = results# 使用utils中的对比函数plot_comparison_summary(comparison_results)print("\n说明 / Notes:")print("- RNN_1layer: 单层RNN，参数少，训练快")print("- RNN_2layers: 双层RNN，建模能力更强")print("- BiRNN: 双向RNN，能够捕获前向和后向依赖")